In [ ]:
__nbid__ = '0001'
__author__ = 'Felix Pat <felixpat@berkeley.edu>'
__version__ = '20251014' # yyyymmdd
__datasets__ = ['Pristine', 'SCSHD', 'SCSLD', 'SCS88in']
__keywords__ = ['Angular Dispersion', 'Principal Component Analysis', 'Contrast Limited Adaptive Histogram Equalization', 'Fast Fourier Transform']

#  Eﬀects of Fission versus Fusion-like Neutron Spectra on REBCO Activation and Microstructure: TEM Analysis
*Author(s): Felix Pat (Univ. of California, Berkeley & Urenco USA)*

### Table of Contents
* [Goal](#Ngoal)
* [Summary](#Nsummary)
* [Disclaimer & Attribution](#Ndisclaimer)
* [Imports & Setup](#Nimport)
* [Read and Preprocess TEM Images](#N0)
* [PCA Analysis for Angular Dispersion Distribution](#N1)
* [Filter Angular Dispersion Distribution](#N2)
* [Plot Angular Dispersion versus Irradiation Dose Boxplots](#N3)
* [References](#N4)
* [Appendix](#N5)

<a class="anchor" id="Ngoal"></a>
#  Goal
This notebook aims to demonstrate a step-by-step workflow from preprocessing TEM images to quantifying the angular dispersion with respect to irradiation dose.

<a class="anchor" id="Nsummary"></a>
#  Summary
This notebook demonstrates how to generally:
1. Read TEM image files in the forms of real space (.dm3), FFT (.dm5), and masked array inverse FFTs TIF images. Default image sizes are 1024x1024, 512x512, and 384x384
2. Standardize and save all PNG images to 6000x6000 while applying an image equalization method, Contrast Limited Adaptive Histogram Equalization, to increase the brightness.
3. 
**For** each **image** in directory:
    - Read image_path.png in binary gray scale 
    - Apply Otsu's thresholding 
    - Segment contours and apply Border Following Algorithm \cite{} via Teh-Chin chain approximation algorithm with a full family hierarchy list tree 
    - **For** each **contour** in image above minimum size (hyperparameter) 
        - Apply Principal Component Analysis for the primary component's eigenvector and eigenvalue 
        - Calculate the angular dispersion $\theta$ of PCA eigenvector via 2-argument arctangent $\theta$ $\epsilon$ [-$\pi$, $\pi$] 
    - **End For**
    - Modulo $\pi$ angular dispersion $\theta$ to model anisotropic behavior such that $\theta$ $\epsilon$ [0, $\pi$] 
    - Filter the bottom 40th percentile of angular dispersion $\theta$ distribution for pristine images only 
    - Filter angular dispersion $\theta$ distribution by the 70th percentile to 97th percentile contours by area, which correlates with the dominant eigenvectors sorted by large PCA eigenvalues 
    - Apply a multi-gaussian fit (try curve_fit else norm.fit) on each images' angular dispersion distribution  

**End For** <br>
4. Plot boxplots of each datasets' angular dispersion versus irradiation dose

**Note:** Order of pathways to image in directory differs depending on how it is read! This will change your labeling. Match output.txt file to your glob(image_directory) to ensure your angles.npy order matches the correct image

<a class="anchor" id="Ndisclaimer"></a>
#  Disclaimer & Attribution

Disclaimers
-----------
Note that using the following work constitutes your agreement with our minimal [Disclaimers]().

If you use this work in your published research, **please cite** the following paper(s):

Acknowledgments
---------------
The work done at the University of California, Berkley, was supported by the U.S. Department of Energy
(DOE) through the Oﬃce of Fusion Energy Sciences under contract to the University of California, Santa
Barbra DOE-Fusion (No. DE-FG02-94ER54275). The work at the UC Berkeley’s College of Engineering was
supported by the Fung Institute for Engineering Leadership’s Master of Engineering degree program,
whose capstone component provided the student team that spearheaded this endeavor. Here, the
authors would like to thank [put the other teammates]. The authors would also like to thank Andrew
Gubser and Jeﬀrey Bickel for technical support. The previous work done by our colleagues in Japan was
performed under the GIMRT Program of the Institute for Materials Research, Tohoku University
(Proposal Nos. 202012-IRKMA-0051 and 202212-IRKMA-0507). The previous work was supported in part
by JSPS KAKENHI Grant Nos. JP16H06008, JP18KK0087, JP23H03665, and JP23K28354. The work at LBNL,
including 88-Inch Cyclotron, Berkeley Center for Magnet Technology, and the National Center for
Electron Microscopy at the Molecular Foundry, was supported by the Director, Oﬃce of Sciences, Oﬃce
of High Energy Physics, Oﬃce of Fusion Energy Sciences, and Oﬃce of Basic Energy Sciences, of the U.S.
Department of Energy under Contract No. DE-AC02-05CH11231 and through a US–Japan High Energy
Physics Collaboration grant. The experimental work at the 88-Inch Cyclotron was additionally supported
by the U.S. Nuclear Data Program in the DOE Oﬃce of Nuclear Physics.

<a class="anchor" id="Nimport"></a>
#  Imports and setup

In [ ]:
# conda env
# ipython == 8.15.0
# python == 3.9.18

# std lib
import os
import copy
from math import atan2, cos, sin, sqrt, pi
import glob

# 3rd party with versions listed
import numpy as np                              # 1.24.0
from scipy.stats import norm                    # 1.13.1
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import torch                                    # 2.1.0
import cv2                                      # 3.4.18.65
import ncempy                                   # 1.11.1
import h5py                                     # 3.10.0
import tifffile                                 # 2024.2.12

# visualization purposes
from typing import List
import numpy.typing as npt
import matplotlib as mpl                        # 3.9.4
from matplotlib import pyplot as plt
from bokeh.plotting import ColumnDataSource, figure, output_file, show, output_notebook

mpl.rcParams['figure.dpi'] = 500 # 200 to read in notebook, 300 for final saving of plots
%matplotlib inline
output_notebook()

<a class="anchor" id="N0"></a>
#  Read Directory Paths with iFFT TEM TIF Images

In [ ]:
DIR_ARRAY_PRISTINE ='/global/cfs/cdirs/m4361/2025/Array/prist/*.tif'
DIR_ARRAY_HD ='/global/cfs/cdirs/m4361/2025/Array/HD/*.tif'
DIR_ARRAY_LD ='/global/cfs/cdirs/m4361/2025/Array/LD/*.tif'
DIR_ARRAY_88 ='/global/cfs/cdirs/m4361/2025/Array/88in/*.tif'

FILE_TYPES = ['.dm3', '.dm5', '.tif', '.png', '.jpg']

def read_file_paths_in_directory(image_directory: str, preprocessed: bool = False) -> List[str]:
    """
    Retrieve the (image) file pathways given a directory

    Parameters
    ----------
    image_directory : str
    Path to directory with image files

    preprocessed : bool
    Indicator for what files to read

    Returns
    -------
    paths : List[str]
    List of paths to images found in image directory
    """
    
    paths = []

    for path in glob.glob(image_directory):
        if preprocessed and 'pca' not in path and 'contrast' in path:
            paths.append(path)
        elif not preprocessed and 'pca' not in path and 'contrast' not in path: # personal identifiers to exclude repeats
            paths.append(path)
            
    return paths

In [ ]:
PATHS_PRISTINE = read_file_paths_in_directory(DIR_ARRAY_PRISTINE)
PATHS_HD = read_file_paths_in_directory(DIR_ARRAY_HD)
PATHS_LD = read_file_paths_in_directory(DIR_ARRAY_LD)
PATHS_88 = read_file_paths_in_directory(DIR_ARRAY_88)
print(len(PATHS_PRISTINE), len(PATHS_HD), len(PATHS_LD), len(PATHS_88))

<a class="anchor"></a>
#  Preprocess iFFT TEM TIF Images and Save as PNG

In [ ]:
def save_upscale_image(image_paths: List[str], file_type: str) -> None:
    """
    Given list of pathways, preprocess iFFT TIF images using CLAHE and save at fixed resolution 6000x6000

    Parameters
    ----------
    image_paths : List[str]
    Paths to image files

    file_type : str
    Type of image file in paths to image files

    Returns
    -------
    None
    """
    
    plt.ioff()
    
    for i, path in enumerate(image_paths):
        
        fig, ax = plt.subplots(figsize=(12,12))
        plt.axis('off')
        plt.tight_layout()
        fig.subplots_adjust(0,0,1,1)
        

        clahe = cv2.createCLAHE(clipLimit=8, tileGridSize=(16,16)) # hyperparameters for CLAHE
        ax.imshow(clahe.apply(cv2.normalize(tifffile.imread(path), None, 0, 255, cv2.NORM_MINMAX).astype('uint8'))) # read TIF image in binary

        fig.savefig(path.replace(file_type, '_contrast.png'), bbox_inches='tight', pad_inches = 0)
    
        plt.close()

In [ ]:
save_upscale_image(PATHS_PRISTINE, FILE_TYPES[2])
save_upscale_image(PATHS_HD, FILE_TYPES[2])
save_upscale_image(PATHS_LD, FILE_TYPES[2])
save_upscale_image(PATHS_88, FILE_TYPES[2])

<a class="anchor" id="N1"></a>
#  PCA Analysis for Angular Dispersion Distribution

In [ ]:
def get_orientation(points: npt.NDArray[float], img: npt.NDArray[float]) -> float:
    """
    Given points of a contour, calculate the PCA eigenvector and eigenvalue and 
    use atan2(eigenvector) to calculate orientation of contour i.e. our dispersion angle

    Parameters
    ----------
    points : npt.NDArray[float]
    Points of contour from cv2.findContours
    
    img : npt.NDArray[float]
    Input TEM image

    Returns
    -------
    angle : float
    Dispersion angle in radians from atan2
    """

    contour_size = len(points)
    data_points = np.empty((contour_size, 2), dtype=np.float64)
    for i in range(data_points.shape[0]):
        data_points[i,0] = points[i,0,0]
        data_points[i,1] = points[i,0,1]

    #  Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_points, mean)

  
    center = (int(mean[0,0]), int(mean[0,1]))
    cv2.circle(img, center, 5, (255, 0, 255), 2) # draw a circle on center of contour
   
    scaled_eigenvector = (center[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], center[1] + 0.02 *  eigenvectors[0,1] * eigenvalues[0,0])
    draw_axis(img, center, scaled_eigenvector, (255, 0 , 0), 1) # draw eigenvector of contour

    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) #  orientation in radians

    return angle
    
def draw_axis(img: npt.NDArray[float], center_: tuple[float, float], eigenvector_: tuple[float, float], colour: tuple[float, float, float], scale: int) -> None:
    """
    Given eigenvector of contour, draw the eigenvector on input TEM image

    Parameters
    ----------
    img : npt.NDArray[float]
    Input TEM image

    center_ : tuple[float, float]
    Center coordinate of contour

    eigenvector_ : tuple[float, float]
    Eigenvector coordinate of contour

    colour : tuple[float, float, float]
    Set eigenvector draw color parameter
    
    scale : int
    Size of eigenvector to draw

    Returns
    -------
    None
    """
    
    center = list(center_)
    eigenvector = list(eigenvector_)

    angle = atan2(center[1] - eigenvector[1], center[0] - eigenvector[0]) #  angle in radians
    hypotenuse = sqrt((center[1] - eigenvector[1]) * (center[1] - eigenvector[1]) + (center[0] - eigenvector[0]) * (center[0] - eigenvector[0]))

    #  Scale and draw arrow
    eigenvector[0] = center[0] - scale * hypotenuse * cos(angle)
    eigenvector[1] = center[1] - scale * hypotenuse * sin(angle)
    cv2.line(img, (int(center[0]), int(center[1])), (int(eigenvector[0]), int(eigenvector[1])), colour, 10, cv2.LINE_AA)

    #  Draw arrow hooks
    center[0] = eigenvector[0] + 9 * cos(angle + pi / 4)
    center[1] = eigenvector[1] + 9 * sin(angle + pi / 4)
    cv2.line(img, (int(center[0]), int(center[1])), (int(eigenvector[0]), int(eigenvector[1])), colour, 10, cv2.LINE_AA)
    center[0] = eigenvector[0] + 9 * cos(angle - pi / 4)
    center[1] = eigenvector[1] + 9 * sin(angle - pi / 4)
    cv2.line(img, (int(center[0]), int(center[1])), (int(eigenvector[0]), int(eigenvector[1])), colour, 10, cv2.LINE_AA)

In [ ]:
def PCA_analysis(directory: List[str], minimum_area: float = 1e3) -> None:
    """
    Main function that performs PCA analysis given directory list of preprocessed TEM images
    and saves .npy angular dispersion distribution and contour areas of each image with
    annotated grayscale image of eigenvector and contours

    Parameters
    ----------
    directory : List[str]
    Path of each class of TEM image directory


    minimum_area : float
    Filter for minimum contour area default to 1000 pixels

    Returns
    -------
    None
    """

    store_angle, store_area, paths = [], [], []

    image_count = 0
    
    for dir in directory:
        for path in glob.glob(dir):

            if 'contrast' in path and 'pca' not in path: # select preprocessed PNG images
                paths.append(path)
                image_count+=1
       
        with open("PCA_output.txt", "a") as f:
            print(len(paths), file=f)
        print(len(paths)) # print number of image paths read from each directory
    
    # print total number of TEM image paths
    with open("PCA_output.txt", "a") as f:
        print(len(paths), file=f)
    print(len(paths))
    for i, path in enumerate(paths):
        
        image_source = cv2.imread(path)

        #  convert to grayscale
        image_gray = cv2.cvtColor(image_source, cv2.COLOR_BGR2GRAY)
        # final image used to draw and save
        image_final = cv2.cvtColor(image_gray, cv2.COLOR_GRAY2BGR)
    
        #  convert img into binary with Otsu's thresholding
        _, image_threshold = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        #  calculate points of contours
        _, contours, _ = cv2.findContours(image_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1) # methods = [CHAIN_APPROX_NONE, CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_TC89_L1, cv2.CHAIN_APPROX_TC89_KCOS]
    
        image_angles = []
        contour_areas = []
        
        for j,c in enumerate(contours):
          #  area of each contour
          area = cv2.contourArea(c)
            
          if area < minimum_area:
            continue

          #  draw each contour on grayscale image
          cv2.drawContours(image_final, contours, j, (0, 0, 255), 2)

          #  find orientation of each shape
          image_angles.append(get_orientation(c,image_final))
          contour_areas.append(area)

        store_angle = store_angle + [image_angles]
        store_area = store_area + [contour_areas]
    
        with open("PCA_output.txt", "a") as f:
            print(len(image_angles), len(contour_areas), path, file=f)
            
        fig,ax=plt.subplots(figsize=(20,20))
        ax.imshow(image_final)
        ax.axis('off')
        fig.savefig(path.replace(FILE_TYPES[3],'') + '_pca_final_%i.png'%i, bbox_inches = 'tight', pad_inches = 0)
        plt.close()

    #  indices separating classes stored in output txt file
    a = np.array(store_angle, dtype=object)
    np.save('pca_final_angle%d.npy'%len(paths), a)
    b = np.array(store_area, dtype=object)
    np.save('pca_final_area%d.npy'%len(paths), b)
    f.close()


In [ ]:
# Note: Reading in PNG will result in different order pathways compared to TIF, which is important to know for saved angles.npy order
PCA_DIRECTORIES = ['/global/cfs/cdirs/m4361/2025/Array/prist/*.png','/global/cfs/cdirs/m4361/2025/Array/HD/*.png','/global/cfs/cdirs/m4361/2025/Array/LD/*.png','/global/cfs/cdirs/m4361/2025/Array/88in/*.png']
    
PCA_analysis(PCA_DIRECTORIES)

<a class="anchor" id="N2"></a>
#  Filter Angular Dispersion Distribution

In [ ]:
# Update order of pathways to match order of angle array
DIR_ARRAY_PRISTINE ='/global/cfs/cdirs/m4361/2025/Array/prist/*.png'
DIR_ARRAY_HD ='/global/cfs/cdirs/m4361/2025/Array/HD/*.png'
DIR_ARRAY_LD ='/global/cfs/cdirs/m4361/2025/Array/LD/*.png'
DIR_ARRAY_88 ='/global/cfs/cdirs/m4361/2025/Array/88in/*.png'

PATHS_PRISTINE = read_file_paths_in_directory(DIR_ARRAY_PRISTINE, preprocessed=True)
PATHS_HD = read_file_paths_in_directory(DIR_ARRAY_HD, preprocessed=True)
PATHS_LD = read_file_paths_in_directory(DIR_ARRAY_LD, preprocessed=True)
PATHS_88 = read_file_paths_in_directory(DIR_ARRAY_88, preprocessed=True)
print(len(PATHS_PRISTINE), len(PATHS_HD), len(PATHS_LD), len(PATHS_88))

In [ ]:
PRISTINE_IDX = len(PATHS_PRISTINE)
HD_IDX = PRISTINE_IDX + len(PATHS_HD)
LD_IDX = HD_IDX + len(PATHS_LD)


ANGLES_PRISTINE = np.load('/global/cfs/cdirs/m4361/2025/analysis/pcafinal_angle416.npy',allow_pickle=True)[:PRISTINE_IDX]
ANGLES_HD = np.load('/global/cfs/cdirs/m4361/2025/analysis/pcafinal_angle416.npy',allow_pickle=True)[PRISTINE_IDX:HD_IDX]
ANGLES_LD = np.load('/global/cfs/cdirs/m4361/2025/analysis/pcafinal_angle416.npy',allow_pickle=True)[HD_IDX:LD_IDX]
ANGLES_88 = np.load('/global/cfs/cdirs/m4361/2025/analysis/pcafinal_angle416.npy',allow_pickle=True)[LD_IDX:]

WEIGHTS_PRISTINE = np.load('/global/cfs/cdirs/m4361/2025/analysis/pcafinal_area416.npy',allow_pickle=True)[:PRISTINE_IDX]
WEIGHTS_HD = np.load('/global/cfs/cdirs/m4361/2025/analysis/pcafinal_area416.npy',allow_pickle=True)[PRISTINE_IDX:HD_IDX]
WEIGHTS_LD = np.load('/global/cfs/cdirs/m4361/2025/analysis/pcafinal_area416.npy',allow_pickle=True)[HD_IDX:LD_IDX]
WEIGHTS_88 = np.load('/global/cfs/cdirs/m4361/2025/analysis/pcafinal_area416.npy',allow_pickle=True)[LD_IDX:]

len(ANGLES_PRISTINE),len(ANGLES_HD),len(ANGLES_LD),len(ANGLES_88)

In [ ]:
def label_filter(angles: npt.NDArray[object], paths: List[str]) -> dict[str, List[float]]:
    """
    Ensure order of angles and paths correspond to the correct image, filter angular
    dispersion distributions by labels: magnification and where TEM image was taken

    Parameters
    ----------
    angles : npt.NDArray[object]
    Angular disperesion distribution of each image

    paths : List[str]
    List of paths to images found in image directory

    Returns
    -------
    angle_dict : dict[str, List[float]]
    Angular dispersion distributions labeled by magnification and where TEM was taken
    """

    mag = ['26k', '38k', '43k', '66k', '92k']
    where = ['edge', 'BZO']

    angle_dict = {
        'edge_26k': [],
        'edge_38k': [],
        'edge_43k': [],
        'edge_66k': [],
        'edge_92k': [],
        'rs_26k': [],
        'rs_38k': [],
        'rs_43k': [],
        'rs_66k': [],
        'rs_92k': [],
        'bzo_26k': [],
        'bzo_38k': [],
        'bzo_43k': [],
        'bzo_66k': [],
        'bzo_92k': [],
    }

    for i in range(len(angles)):
        if mag[0] in paths[i] and where[0] in paths[i]:
            angle_dict['edge_26k'].append(i)
        elif mag[1] in paths[i] and where[0] in paths[i]:
            angle_dict['edge_38k'].append(i)
        elif mag[2] in paths[i] and where[0] in paths[i]:
            angle_dict['edge_43k'].append(i)
        elif mag[3] in paths[i] and where[0] in paths[i]:
            angle_dict['edge_66k'].append(i)
        elif mag[4] in paths[i] and where[0] in paths[i]:
            angle_dict['edge_92k'].append(i)

        elif mag[0] in paths[i] and where[1] in paths[i]:
            angle_dict['bzo_26k'].append(i)
        elif mag[1] in paths[i] and where[1] in paths[i]:
            angle_dict['bzo_38k'].append(i)
        elif mag[2] in paths[i] and where[1] in paths[i]:
            angle_dict['bzo_43k'].append(i)
        elif mag[3] in paths[i] and where[1] in paths[i]:
            angle_dict['bzo_66k'].append(i)
        elif mag[4] in paths[i] and where[1] in paths[i]:
            angle_dict['bzo_92k'].append(i)
            
        elif mag[0] in paths[i]:
            angle_dict['rs_26k'].append(i)
        elif mag[1] in paths[i]:
            angle_dict['rs_38k'].append(i)
        elif mag[2] in paths[i]:
            angle_dict['rs_43k'].append(i)
        elif mag[3] in paths[i]:
            angle_dict['rs_66k'].append(i)
        elif mag[4] in paths[i]:
            angle_dict['rs_92k'].append(i)
   
    for i, val in enumerate(angle_dict.values()):
        
        if i % 5 == 0:
            print()
        print(len(val), end=' ')
        
    print()
        
    return angle_dict

In [ ]:
ANGLES_PRISTINE_LABELED = label_filter(ANGLES_PRISTINE, PATHS_PRISTINE)
ANGLES_HD_LABELED = label_filter(ANGLES_HD, PATHS_HD)
ANGLES_LD_LABELED = label_filter(ANGLES_LD, PATHS_LD)
ANGLES_88_LABELED = label_filter(ANGLES_88, PATHS_88)

""" 
print output in (row, column) format
            26k 38k 43k 66k 92k
Edge
Real Space
BZO
"""

In [ ]:
def modulo_pi(angles: npt.NDArray[object]) -> npt.NDArray[object]:
    """
    Modulo pi angular dispersion disrtribution such that angles range 0 to pi

    Parameters
    ----------
    angles : npt.NDArray[object]
    Angular dispersion distribution of each image

    Returns
    -------
    angles : npt.NDArray[object]
    Angular dispersion distribution of each image
    """
    
    for i in range(len(angles)):
        angles[i] = [(k + np.pi if k <= 0. else k) for k in angles[i]]

    return angles

In [ ]:
def weight_filter(angles: npt.NDArray[object], weights: npt.NDArray[object], upper_percentile: float = 97, pristine: bool = False) -> npt.NDArray[object]:
    """
    Filter angular dispersion distribution by contour size i.e. eigenvalue

    Parameters
    ----------
    angles: npt.NDArray[object]
    Angular dispersion distribution of each image

    weights: npt.NDArray[object]
    Contour areas of each angle of each image

    upper_percentile: float = 97
    Upper percentile filter on contour areas

    pristine : bool
    Input data is pristine image data or not

    Returns
    -------
    angles_filtered : npt.NDArray(object)
    Angular dispersion distribution after contour size filters
    """
   
    angles_filtered = angles.copy()

    for i, angle in enumerate(angles):
        
        threshold1 = np.percentile(weights[i], upper_percentile)
        threshold2 = np.percentile(weights[i], 70)
        
        if pristine:
            threshold3 = np.percentile(angle, 40)
            mask = (weights[i] >= threshold2) & (weights[i] <= threshold1) & (angle > threshold3)
            
        else:
            mask = (weights[i] >= threshold2) & (weights[i] <= threshold1)

        angles_filtered[i] = np.array(angle)[mask]
  
    return angles_filtered

In [ ]:
ANGLES_PRISTINE = modulo_pi(ANGLES_PRISTINE)
ANGLES_HD = modulo_pi(ANGLES_HD)
ANGLES_LD = modulo_pi(ANGLES_LD)
ANGLES_88 = modulo_pi(ANGLES_88)

ANGLES_PRISTINE_FILTERED = weight_filter(ANGLES_PRISTINE, WEIGHTS_PRISTINE, pristine = True)
ANGLES_HD_FILTERED = weight_filter(ANGLES_HD, WEIGHTS_HD)
ANGLES_LD_FILTERED = weight_filter(ANGLES_LD, WEIGHTS_LD)
ANGLES_88_FILTERED = weight_filter(ANGLES_88, WEIGHTS_88)

<a class="anchor" id="N3"></a>
# Plot Boxplots Angular Dispersion versus Irradiation Dose

In [ ]:
def multi_gaussian(x: float, *params: float) -> float:
    """
    Gaussian distribution function for curve_fit

    Parameters
    ----------
    x : float
    Data point at x

    Returns
    -------
    y : float
    Data point estimated at y
    """
    
    y = np.zeros_like(x)

    for i in range(len(params) // 3):
        amp, mean, std = params[3*i:3*i+3]
        y += amp * np.exp(-((x - mean)**2) / (2 * std**2))

    return y

def guess_initial_params_uniform(x: List[float], y: List[float], n_components: int) -> List[float]:
    """
    Setup curve_fit initial guess for multigaussian distribution

    Parameters
    ----------
    x : float
    Data point at x

    y : float
    Data point at y

    n_components : int
    N number of initial guesses at different means which will output N number of gaussians

    Returns
    -------
    init_params : List[float]
    Gaussian distribution initial guess of parameters
    """

    amp_guess = max(y) / n_components
    mean_guess = np.linspace(min(x), max(x), n_components)
    if n_components == 1 and min(x) < 3/4 * np.pi and max(x) > 3/4 * np.pi:
        mean_guess = np.array([3/4 * np.pi])
    std_guess = [(max(x) - min(x)) / (2 * n_components)] * n_components
    init_params = []

    for a, m, s in zip([amp_guess]*n_components, mean_guess, std_guess):
        init_params.extend([a, m, s])

    return init_params

def fit_multi_gaussian(x: List[float], y: List[float], n_components: int = 3) -> List[float]:
    """
    Setup curve_fit initial guess for multigaussian distribution

    Parameters
    ----------
    x : float
    Data point at x

    y : float
    Data point at y

    n_components : int
    N number of initial guesses at different means which will output N number of gaussians

    Returns
    -------
    init_params : List[float]
    Gaussian distribution initial guess of parameters
    """
    
    lower_bounds = []
    upper_bounds = []
    
    for i in range(n_components):
        lower_bounds.extend([0, min(x), 0.01])  #  amp ≥ 0, mean ≥ min(x), std ≥ 0.01
        upper_bounds.extend([max(y)*2, max(x), np.pi/2])  #  std can span full range

    init_params = guess_initial_params_uniform(x, y, n_components)
    fit_params, _ = curve_fit(multi_gaussian, x, y, p0=init_params, bounds=(lower_bounds, upper_bounds))

    return fit_params

def plot_hists(image_angles: npt.NDArray[object], components : int = 3) -> tuple[List[float], List[float]]:
    """
    Retrieve the 

    Parameters
    ----------
    image_angles : npt.NDArray[object]
    Angular dispersion distribution of each image

    components : int
    N number of multi-gaussians to fit

    Returns
    -------
    store_mean : List[float]
    store_sigma : List[float]
    """
    store_mean, store_sigma = [], []

    # calculate number of rows and columns needed to draw all distributions with upper limit
    num = int(len(image_angles)**(1/2)) + 1
    print(num)

    # if desired, limit number of plots drawn to reduce loading time
    #if num > 5:
    #    num = 5

    fig, ax = plt.subplots(num, num, figsize=(20,20))

    # idx used to ID image path as needed
    count = 0

    for i in range(num):
        for j in range(num):
    
            n, bins, patches = ax[i,j].hist(image_angles[count], 100, density=True, facecolor='green', alpha=0.75)

            # if curve_fit params explodes to infinity, use norm.fit for normal distribution approximation
            try:
                bins_multi = (bins[:-1] + bins[1:]) / 2
                fit_params = fit_multi_gaussian(bins_multi, n, n_components=components)
    
                means = fit_params[1::3].tolist()
                stds = fit_params[2::3].tolist()
                
                store_mean = store_mean + means
                store_sigma = store_sigma + stds
                
                x_min = min(means) - 3 * max(stds)
                x_max = max(means) + 3 * max(stds)
                x = np.linspace(x_min, x_max, 100)
            
                for k in range(len(fit_params) // 3):
                    amp, mean, std = fit_params[3*k:3*k+3]
                    g = amp * np.exp(-((x - mean)**2) / (2 * std**2))
                    ax[i,j].plot(x, g, '--', c='red', label=f'Gaussian {k+1}', zorder=4)

            except:
                (mu, sigma) = norm.fit(np.array(image_angles[count], dtype = float))
                #  add a 'best fit' line
                y = norm.pdf( bins, mu, sigma)
                l = ax[i,j].plot(bins, y, 'r--', linewidth=2)
                store_mean = store_mean + [mu]
                store_sigma = store_sigma + [sigma]
                
            ax[i,j].set_xlabel('Angle (radians)',fontsize=20)
            ax[i,j].tick_params(
                axis='y',          #  changes apply to the y-axis
                which='both',      #  both major and minor ticks are affected
                left=False,      #  ticks along the bottom edge are off
                right=False,         #  ticks along the top edge are off
                labelleft=False) #  labels along the bottom edge are off
            # ax[i,j].ylabel('Count',fontsize=20)
            ax[i,j].text(.01, .99, '%i'%(count), ha='left', va='top', transform=ax[i,j].transAxes,fontsize=20)
            ax[i,j].set_xlim(0,np.pi)
            count+=1
            plt.tight_layout()

            if count == len(image_angles):
                break
        if count == len(image_angles):
            break
    
    plt.show()
    
    return store_mean, store_sigma

In [ ]:
_, SIGMA_PRISTINE = plot_hists(ANGLES_PRISTINE_FILTERED[ANGLES_PRISTINE_LABELED['rs_66k']], components=1)

_, SIGMA_HD = plot_hists(ANGLES_HD_FILTERED[ANGLES_HD_LABELED['rs_66k']], components=1)

_, SIGMA_LD =  plot_hists(ANGLES_LD_FILTERED[ANGLES_LD_LABELED['rs_66k']], components=1)

_, SIGMA_88 = plot_hists(ANGLES_88_FILTERED[ANGLES_88_LABELED['rs_66k']], components=1)

In [ ]:
def plot_boxplot(data: List[npt.NDArray[float]]) -> None:
    labels = ['Pristine n=%i\n'%len(data[0]) + r'0.0 $\dfrac{n}{m^{2}}$',
              'BR2 Low Dose n=%i\n'%len(data[1]) + r'$8.23 \times 10^{21} \dfrac{n}{m^{2}}$',
              'BR2 High Dose n=%i\n'%len(data[2]) + r'$7.92 \times 10^{22} \dfrac{n}{m^{2}}$',
              '88-Inch Cyclotron n=%i\n'%len(data[3]) + r'$1.0 \times 10^{16} \dfrac{n}{m^{2}}$',]
    
    colors = ['#003262', (0,176/255,80/255), 'red', '#FDB515']
    mag = ['26k', '38k', '43k', '66k', '92k']
    
    fig, ax = plt.subplots(figsize=(23,23))
    ax.set_ylabel(r'Angular Dispersion V$\theta$ (Radians)', fontsize=40)
    plt.xticks(fontsize=31)
    plt.yticks(fontsize=30)  
    
    flierprops = dict(marker='o', markerfacecolor='black', markersize=15,
                      linestyle='none', markeredgecolor='black')
    
    
    bplot = ax.boxplot(data,
                        patch_artist=True,
                        flierprops=flierprops,
                        showmeans=True,
                        meanline=True,
                        tick_labels=labels,
                        whis=[0, 100])

    # fill with colors
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)

    for median in bplot['medians']:
        median.set_color('black')
        median.set_linewidth(2)
    for mean in bplot['means']:
        mean.set_color('black')
        mean.set_linewidth(2)
        
    stats_text = []
    for label, category_data in zip(labels, data):
        mean_val = np.mean(category_data)
        median_val = np.median(category_data)
        stats_text.append(f'$\mu$: {mean_val:.4f}, Median: {median_val:.4f}')
    
    
    
    # Add the text at y=1.57
    for i in range(1,5):
        ax.text(i, -0.035, stats_text[i-1], ha='center', va='center', fontsize=25, color='black')
    ax.grid(True, axis='y', linestyle='--', linewidth=2)
        
    plt.tight_layout()
    plt.show()
    fig.savefig('/global/cfs/cdirs/m4361/2025/Publication/boxplot_angulardispersion.svg',bbox_inches='tight',pad_inches=0, format='svg')

In [ ]:
BOXPLOT_DATA = [np.array(SIGMA_PRISTINE),
        np.array(SIGMA_LD),
        np.array(SIGMA_HD),
        np.array(SIGMA_88)]

plot_boxplot(BOXPLOT_DATA)

<a class="anchor" id="N4"></a>
#  References

1 [Suzuki, S. and be, K. (1985).](https://doi.org/10.1016/0734-189X(85)90016-7) Topological structural analysis of digitized binary images by border following. Computer Vision, Graphics, and Image Processing, 30(1), pp.32–46. doi:https://doi.org/10.1016/0734-189x(85)90016-7. <br>
2 [Teh, C.-H. . and Chin, R.T. (1989).](https://doi.org/10.1109/34.31447) On the detection of dominant points on digital curves. IEEE Transactions on Pattern Analysis and Machine Intelligence, 11(8), pp.859–872. doi:https://doi.org/10.1109/34.31447. <br>
3 [Otsu, N. (1979).](https://doi.org/10.1109/tsmc.1979.4310076) A Threshold Selection Method from Gray-Level Histograms. IEEE Transactions on Systems, Man, and Cybernetics, [online] 9(1), pp.62–66. doi:https://doi.org/10.1109/tsmc.1979.4310076. <br>
4 [Zuiderveld, K. (1994).](https://doi.org/10.1016/b978-0-12-336156-1.50061-6) Contrast Limited Adaptive Histogram Equalization. Graphics Gems, pp.474–485. doi:https://doi.org/10.1016/b978-0-12-336156-1.50061-6. <br>
5 [Jolliffe, I.T. (2002).](https://doi.org/10.1007/b98835) Principal Component Analysis. [online] Springer Series in Statistics. New York: Springer-Verlag. doi:https://doi.org/10.1007/b98835. <br>
6 [Bradski, G. (2000).](https://github.com/opencv/opencv-python) The OpenCV Library. Dr. Dobb's Journal of Software Tools. <br>
7 [Virtanen, P. et al. (2020).](https://github.com/scipy/scipy) SciPy 1.0: Fundamental Algorithms for Scientific Computing in 
Python. Nature Methods, 17, pp.261–272. <br>


<a class="anchor" id="N5"></a>
#  Appendix

Workflow Plots

In [ ]:
IMG_PRISTINE = np.array(PATHS_PRISTINE)[ANGLES_PRISTINE_LABELED['rs_66k']][1]
IMG_HD = np.array(PATHS_HD)[ANGLES_HD_LABELED['rs_66k']][1]
IMG_LD = np.array(PATHS_LD)[ANGLES_LD_LABELED['rs_66k']][8]
IMG_88 = np.array(PATHS_88)[ANGLES_88_LABELED['rs_66k']][1]
print(IMG_PRISTINE, IMG_HD, IMG_LD, IMG_88)

In [ ]:
def find_idx(img_path, paths: str) -> int:
    
    i=0
    for p in paths:
        if p == img_path:
            return i
        i+=1

In [ ]:
IDX_P = find_idx(IMG_PRISTINE, PATHS_PRISTINE)
IDX_HD = find_idx(IMG_HD, PATHS_HD) + PRISTINE_IDX
IDX_LD = find_idx(IMG_LD, PATHS_LD) + HD_IDX
IDX_88 = find_idx(IMG_88, PATHS_88) + LD_IDX
IDX_P, IDX_HD, IDX_LD, IDX_88

In [ ]:
select_paths = ['/global/cfs/cdirs/m4361/2025/Array/prist/33_66k.tif', 
                '/global/cfs/cdirs/m4361/2025/Array/HD/296_66k.tif',
                '/global/cfs/cdirs/m4361/2025/Array/LD/155_66k.tif',
                '/global/cfs/cdirs/m4361/2025/Array/88in/388_66k.tif']

for i, path in enumerate(select_paths):
    
    fig, ax = plt.subplots(figsize=(12,12))
    plt.axis('off')
    plt.tight_layout()
    fig.subplots_adjust(0,0,1,1)
    
    ax.imshow(tifffile.imread(path), cmap='gray') # read TIF image in binary

    fig.savefig(path.replace(FILE_TYPES[2], '.svg'), bbox_inches='tight', pad_inches = 0, format='svg')
    plt.show()
    plt.close()

In [ ]:
def plot_image(path: str) -> None:
    fig, ax = plt.subplots(figsize=(12,12))
    plt.axis('off')
    plt.tight_layout()
    fig.subplots_adjust(0,0,1,1)
    plt.imshow(cv2.imread(path))
    
    plt.show()
    fig.savefig(path.replace('.png', '.svg'), bbox_inches='tight', pad_inches = 0, format='svg')

In [ ]:
plot_image(IMG_PRISTINE.replace('.png', '_pca_final_%i.png'%IDX_P))

In [ ]:
plot_image(IMG_HD.replace('.png', '_pca_final_%i.png'%IDX_HD))

In [ ]:
plot_image(IMG_LD.replace('.png', '_pca_final_%i.png'%IDX_LD))

In [ ]:
plot_image(IMG_88.replace('.png', '_pca_final_%i.png'%IDX_88))

In [ ]:
def plot_hist(data: npt.NDArray[float], path: str, text: str = '') -> None:
    fig,ax=plt.subplots(figsize=(10,10))
    
    n, bins, patches = ax.hist(data, 100, density=True, facecolor='green', edgecolor='black', alpha=0.75)
    
    # add a 'best fit' line
    bins_multi = (bins[:-1] + bins[1:]) / 2
    fit_params = fit_multi_gaussian(bins_multi, n, n_components=1)
    
    means = fit_params[1::3].tolist()
    stds = fit_params[2::3].tolist()
    
    x_min = min(means) - 3 * max(stds)
    x_max = max(means) + 3 * max(stds)
    x = np.linspace(x_min, x_max, 100)
    
    for k in range(len(fit_params) // 3):
        amp, mean, std = fit_params[3*k:3*k+3]
        g = amp * np.exp(-((x - mean)**2) / (2 * std**2))
        ax.plot(x, g, '--', c='red', label=f'Gaussian {k+1}', zorder=4, linewidth=7)
        
    ax.set_xlabel(r'Angular Dispersion V$\theta$ (Radians)', fontsize=30)
    ax.text(.01, .99, text + r': $\mu$=%.4f, $\sigma$=%.4f'%(means[0], stds[0]), ha='left', va='top', transform=ax.transAxes,fontsize=28)
    plt.xticks(fontsize=25)
    ax.tick_params(axis='y', which='both', length=0)
    ax.set_yticklabels([])
    ax.set_xlim(0,np.pi)
    plt.tight_layout()
    ax.grid(True, axis='x', linestyle='--', linewidth=2)
    plt.show()
    fig.savefig(path.replace('.tif', '_anghist.svg'), bbox_inches='tight', pad_inches = 0, format='svg')

In [ ]:
plot_hist(ANGLES_PRISTINE_FILTERED[ANGLES_PRISTINE_LABELED['rs_66k']][1], path = select_paths[0], text='Pristine')

In [ ]:
plot_hist(ANGLES_HD_FILTERED[ANGLES_HD_LABELED['rs_66k']][1], path = select_paths[1], text='HD')

In [ ]:
plot_hist(ANGLES_LD_FILTERED[ANGLES_LD_LABELED['rs_66k']][8], path = select_paths[2], text='LD')

In [ ]:
plot_hist(ANGLES_88_FILTERED[ANGLES_88_LABELED['rs_66k']][1], path = select_paths[3], text='88 Inch')

Read other types of file types

In [ ]:
def read_dm3(path: str) -> tuple[npt.NDArray[float], float, float]:
    """
    Retrieve the 

    Parameters
    ----------
    image_directory : str
    Path to directory with image files

    Returns
    -------
    paths : List[str]
    List of paths to images found in image directory
    """
    dmfile = ncempy.io.dm.fileDM(path)
    tags = dmfile.allTags

    lower_bound = tags['.DocumentObjectList.1.ImageDisplayInfo.LowLimit']
    upper_bound = tags['.DocumentObjectList.1.ImageDisplayInfo.HighLimit']
    ax1_scale = tags['.ImageList.2.ImageData.Calibrations.Dimension.1.Scale']
    ax1_units = tags['.ImageList.2.ImageData.Calibrations.Dimension.1.Units']
    ax2_scale = tags['.ImageList.2.ImageData.Calibrations.Dimension.2.Scale']
    ax2_units = tags['.ImageList.2.ImageData.Calibrations.Dimension.2.Units']

    return dmfile.getDataset(0)['data'], lower_bound, upper_bound

def threshold_dm3(data: np.ndarray, lower_bound: float, upper_bound:float, scale: bool = False) -> np.ndarray:
    """
    Retrieve the 

    Parameters
    ----------
    image_directory : str
    Path to directory with image files

    Returns
    -------
    paths : List[str]
    List of paths to images found in image directory
    """
    data = data.clip(lower_bound, upper_bound)
    if scale:
        data = (data - lower_bound) / upper_bound * 255
    return data

In [ ]:
def dm3_to_image(paths: str) -> tuple[List[float], List[str], List[float], List[str]]:
    """
    Retrieve the 

    Parameters
    ----------
    image_directory : str
    Path to directory with image files

    Returns
    -------
    paths : List[str]
    List of paths to images found in image directory
    """
    datatensor1024 = torch.tensor([])
    datatensor512 = torch.tensor([])
    path1024 = []
    path512 = []
    count = 0
    
    for i, path in enumerate(paths):
        try:
            data, lower_bound, upper_bound = read_dm3(path)
            data = threshold_dm3(data, lower_bound, upper_bound, scale=True)
        except:
            count += 1
            continue
        if data.shape[0] == 1024 and data.shape[1] == 1024:
            datatensor1024 = torch.cat((datatensor1024, torch.tensor(data)[:,:,None]), -1)
            path1024.append(path)
        if data.shape[0] == 512 and data.shape[1] == 512:
            datatensor512 = torch.cat((datatensor512, torch.tensor(data)[:,:,None]), -1)
            path512.append(path)
    print(count)

    return datatensor1024, path1024, datatensor512, path512

In [ ]:
DATA_PRISTINE1024, PATH_PRISTINE1024, DATA_PRISTINE512, PATH_PRISTINE512 = dm3_to_image(PATHS_PRISTINE)
DATA_HD1024, PATH_HD1024, DATA_HD512, PATH_HD512 = dm3_to_image(PATHS_HD)
DATA_LD1024, PATH_LD1024, DATA_LD512, PATH_LD512 = dm3_to_image(PATHS_LD)
DATA_881024, PATH_881024, DATA_88512, PATH_88512 = dm3_to_image(PATHS_88)

In [ ]:
def dm5_to_image(paths: str) -> tuple[List[float], List[str]]:
    """
    Retrieve the 

    Parameters
    ----------
    image_directory : str
    Path to directory with image files

    Returns
    -------
    paths : List[str]
    List of paths to images found in image directory
    """
    datatensor = torch.tensor([])
    path = []

    error_count = 0
    
    for i, path in enumerate(paths):
        try:
            with h5py.File(path, 'r') as f:
                data = f['ImageList/[0]/ImageData/Data'][()][:,:,0]
        except:
            count += 1
            continue
            
        try:
            datatensor = torch.cat((datatensor, torch.tensor(data)[:,:,None]), -1)
            path.append(path)
        except:
            error_count += 1
       
    print(error_count)

    return datatensor, path